# 제품 이상여부 판별 프로젝트


### 데이터 읽어오기


In [2]:
import pandas as pd

THRESHOLD = 0.3
RANDOM_STATE = 110

train_data = pd.read_csv("../../../data/train_data_0817.csv")
test_data = pd.read_csv("../../../data/test_data_0817.csv")

In [3]:
train_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Data columns (total 38 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   CURE SPEED Collect Result_Dam                   40506 non-null  int64  
 1   DISCHARGED SPEED OF RESIN Collect Result_Dam    40506 non-null  int64  
 2   Head Clean Position Z Collect Result_Dam        40506 non-null  float64
 3   Head Purge Position Z Collect Result_Dam        40506 non-null  float64
 4   Head Zero Position Y Collect Result_Dam         40506 non-null  float64
 5   Stage2_Circle_Distance_Speed_Dam                40506 non-null  int64  
 6   WorkMode Collect Result                         40506 non-null  float64
 7   Chamber Temp. Collect Result_AutoClave          40506 non-null  int64  
 8   DISCHARGED SPEED OF RESIN Collect Result_Fill1  40506 non-null  float64
 9   Head Purge Position Z Collect Result_Fi

In [4]:
# dam, fill1, fill2 공통 변수
var_dam_fill = [
    'Equipment_same_num',
    'PalletID_Collect_Result_encoded',
    'Production_Qty_Collect_Result',
    'WorkMode Collect Result'
]

In [5]:
# 전체 공통 변수
### correlation 확인을 위한 변수 리스트
var_all_corr = [
    'model_receip_encoded',
    'workorder_receip_encoded'
]

### train
var_all_train = [
    'target',
    'model_receip_encoded',
    'workorder_receip_encoded'
]

### test
var_all_test = [
    'Set ID',
    'target',
    'model_receip_encoded',
    'workorder_receip_encoded'
]

In [6]:
# '_Dam'을 포함하는 변수 선택
dam_variables = [var for var in train_data.columns if '_Dam' in var]

# train
final_columns_train = var_dam_fill + var_all_train + dam_variables
train_data_dam = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + dam_variables
test_data_dam = test_data[final_columns_test]

In [7]:
# '_Fill1'을 포함하는 변수 선택
fill1_variables = [var for var in train_data.columns if '_Fill1' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill1_variables
train_data_fill1 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill1_variables
test_data_fill1 = test_data[final_columns_test]

In [8]:
# '_Fill2'을 포함하는 변수 선택
fill2_variables = [var for var in train_data.columns if '_Fill2' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill2_variables
train_data_fill2 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill2_variables
test_data_fill2 = test_data[final_columns_test]

In [9]:
# '_AutoClave'을 포함하는 변수 선택
autoclave_variables = [var for var in train_data.columns if '_AutoClave' in var]

# train
final_columns_train = var_all_train + autoclave_variables
train_data_autoclave = train_data[final_columns_train]

# test 
final_columns_test = var_all_test + autoclave_variables
test_data_autoclave = test_data[final_columns_test]

In [10]:
# 각 DataFrame의 칼럼 수 계산
num_columns_train_data = train_data.shape[1]
num_columns_train_data_dam = train_data_dam.shape[1]
num_columns_train_data_autoclave = train_data_autoclave.shape[1]
num_columns_train_data_fill1 = train_data_fill1.shape[1]
num_columns_train_data_fill2 = train_data_fill2.shape[1]

num_columns_test_data = test_data.shape[1]
num_columns_test_data_dam = test_data_dam.shape[1]
num_columns_test_data_autoclave = test_data_autoclave.shape[1]
num_columns_test_data_fill1 = test_data_fill1.shape[1]
num_columns_test_data_fill2 = test_data_fill2.shape[1]

# 각 DataFrame의 칼럼 수 출력
print("----train data-----")
print(f"train_data DataFrame의 칼럼 수: {num_columns_train_data}")
print(f"train_data_dam DataFrame의 칼럼 수: {num_columns_train_data_dam}")
print(f"train_data_autoclave DataFrame의 칼럼 수: {num_columns_train_data_autoclave}")
print(f"train_data_fill1 DataFrame의 칼럼 수: {num_columns_train_data_fill1}")
print(f"train_data_fill2 DataFrame의 칼럼 수: {num_columns_train_data_fill2}")
print("----test data-----")
print(f"test_data DataFrame의 칼럼 수: {num_columns_test_data}")
print(f"test_data_dam DataFrame의 칼럼 수: {num_columns_test_data_dam}")
print(f"test_data_autoclave DataFrame의 칼럼 수: {num_columns_test_data_autoclave}")
print(f"test_data_fill1 DataFrame의 칼럼 수: {num_columns_test_data_fill1}")
print(f"test_data_fill2 DataFrame의 칼럼 수: {num_columns_test_data_fill2}")

----train data-----
train_data DataFrame의 칼럼 수: 38
train_data_dam DataFrame의 칼럼 수: 21
train_data_autoclave DataFrame의 칼럼 수: 8
train_data_fill1 DataFrame의 칼럼 수: 13
train_data_fill2 DataFrame의 칼럼 수: 13
----test data-----
test_data DataFrame의 칼럼 수: 39
test_data_dam DataFrame의 칼럼 수: 22
test_data_autoclave DataFrame의 칼럼 수: 9
test_data_fill1 DataFrame의 칼럼 수: 14
test_data_fill2 DataFrame의 칼럼 수: 14


---

## Optuna

스레스홀드 0.3으로 맞춘상태에서 튜닝 진행한 것

In [11]:
# import optuna
# from lightgbm import LGBMClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import f1_score

# # 'Normal'과 'AbNormal'을 숫자로 변환
# train_data_fill1['target'] = train_data_fill1['target'].map({'Normal': 0, 'AbNormal': 1})

# # 스레드홀드 설정
# THRESHOLD = 0.3

# def objectiveLGBM_dart(trial, x_tr, y_tr, x_val, y_val):
#     param = {
#         'n_estimators': trial.suggest_int('n_estimators', 500, 3000),
#         'num_leaves': trial.suggest_int('num_leaves', 500, 3000),
#         'max_depth': trial.suggest_int('max_depth', 10, 300),
#         'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
#         'min_child_samples': trial.suggest_int('min_child_samples', 3, 300),
        
#         'boosting_type': 'dart',  # 'boosting'를 'boosting_type'으로 수정
#         'random_state': RANDOM_STATE,
#         'verbose': -1
#     }
       
#     model = LGBMClassifier(**param)
#     model.fit(x_tr, y_tr)
#     pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
#     pred = (pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측
    
#     score = f1_score(y_val, pred, average="binary")
    
#     return score

# # 데이터셋 분할
# x_train, x_val, y_train, y_val = train_test_split(
#     train_data_fill1.drop("target", axis=1),
#     train_data_fill1["target"],
#     test_size=0.2,
#     shuffle=True,
#     random_state=RANDOM_STATE,
# )

# # 하이퍼 파라미터 튜닝
# study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
# study.optimize(lambda trial: objectiveLGBM_dart(trial, x_train, y_train, x_val, y_val), n_trials=200)

# print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))


Trial 35 finished with value: 0.21714285714285714  
parameters: {'n_estimators': 1399, 'num_leaves': 1456, 'max_depth': 38,  
'learning_rate': 0.0020188481674757053, 'min_child_samples': 21}

In [12]:
# import optuna
# from lightgbm import LGBMClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import f1_score

# # 'Normal'과 'AbNormal'을 숫자로 변환
# train_data_fill1['target'] = train_data_fill1['target'].map({'Normal': 0, 'AbNormal': 1})

# # 스레드홀드 설정
# THRESHOLD = 0.3

# def objectiveLGBM_dart(trial, x_tr, y_tr, x_val, y_val):
#     param = {
#         'n_estimators': trial.suggest_int('n_estimators', 600, 2800),
#         'num_leaves': trial.suggest_int('num_leaves', 1000, 3000),
#         'max_depth': trial.suggest_int('max_depth', 10, 100),
#         'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
#         'min_child_samples': trial.suggest_int('min_child_samples', 3, 100),
        
#         'boosting_type': 'dart',  # 'boosting'를 'boosting_type'으로 수정
#         'random_state': RANDOM_STATE,
#         'verbose': -1
#     }
       
#     model = LGBMClassifier(**param)
#     model.fit(x_tr, y_tr)
#     pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
#     pred = (pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측
    
#     score = f1_score(y_val, pred, average="binary")
    
#     return score

# # 데이터셋 분할
# x_train, x_val, y_train, y_val = train_test_split(
#     train_data_fill1.drop("target", axis=1),
#     train_data_fill1["target"],
#     test_size=0.2,
#     shuffle=True,
#     random_state=RANDOM_STATE,
# )

# # 하이퍼 파라미터 튜닝
# study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
# study.optimize(lambda trial: objectiveLGBM_dart(trial, x_train, y_train, x_val, y_val), n_trials=300)

# print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))


Trial 2 finished with value: 0.2260668973471742  
parameters: {'n_estimators': 1452, 'num_leaves': 1581, 'max_depth': 22, 'learning_rate': 0.002000452888170992, 'min_child_samples': 43}.

한번더 돌렸을때 더 낮은 성능  

In [13]:
import optuna
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# 'Normal'과 'AbNormal'을 숫자로 변환
train_data_fill1['target'] = train_data_fill1['target'].map({'Normal': 0, 'AbNormal': 1})

# 스레드홀드 설정
THRESHOLD = 0.3

def objectiveLGBM_dart(trial, x_tr, y_tr, x_val, y_val):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 3500),
        'num_leaves': trial.suggest_int('num_leaves', 1000, 3000),
        'max_depth': trial.suggest_int('max_depth', 10, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.0003, 0.3),
        'min_child_samples': trial.suggest_int('min_child_samples', 3, 100),
        
        'boosting_type': 'dart',  # 'boosting'를 'boosting_type'으로 수정
        'random_state': RANDOM_STATE,
        'verbose': -1
    }
       
    model = LGBMClassifier(**param)
    model.fit(x_tr, y_tr)
    pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    pred = (pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측
    
    score = f1_score(y_val, pred, average="binary")
    
    return score

# 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    train_data_fill1.drop("target", axis=1),
    train_data_fill1["target"],
    test_size=0.2,
    shuffle=True,
    random_state=RANDOM_STATE,
)

# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
study.optimize(lambda trial: objectiveLGBM_dart(trial, x_train, y_train, x_val, y_val), n_trials=300)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))


C:\Users\juneh\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
[I 2024-08-22 00:23:29,825] A new study created in memory with name: no-name-272167b9-32c5-4090-b176-e6bde723b495
[I 2024-08-22 00:25:14,377] Trial 0 finished with value: 0.19089574155653452 and parameters: {'n_estimators': 848, 'num_leaves': 2317, 'max_depth': 44, 'learning_rate': 0.18395093969213586, 'min_child_samples': 69}. Best is trial 0 with value: 0.19089574155653452.
[I 2024-08-22 00:38:09,779] Trial 1 finished with value: 0.20535714285714288 and parameters: {'n_estimators': 2548, 'num_leaves': 2910, 'max_depth': 67, 'learning_rate': 0.15402919924162803, 'min_child_samples': 11}. Best is trial 1 w

: 

.